In [5]:
import tweepy
from tweepy.auth import OAuthHandler
import pandas as pd
import preprocessor as p
import nltk
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import re

In [8]:
tweets = pd.read_csv('covid_tweets.csv')

In [9]:
def preprocess_data(data):
    #Removes Numbers
    data = data.astype(str).str.replace('\d+', '')
    lower_text = data.str.lower()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer =  TweetTokenizer()
    def lemmatize_text(text):
        return [(lemmatizer.lemmatize(w)) for w \
                       in w_tokenizer.tokenize((text))]
    def remove_punctuation(words):
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', (word))
            if new_word != '':
                new_words.append(new_word)
        return new_words
    words = lower_text.apply(lemmatize_text)
    words = words.apply(remove_punctuation)
    return pd.DataFrame(words)

pre_tweets = preprocess_data(tweets['tweet'])
tweets['tweet'] = pre_tweets
stop_words = set(stopwords.words('english'))
tweets['tweet'] = tweets['tweet'].apply(lambda x: ' '.join([item for item in \
                                    x if item not in stop_words]))

/var/folders/sg/4x_5htrx3y7fm7z77wcz8c_40000gn/T/ipykernel_49760/2874991781.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data = data.astype(str).str.replace('\d+', '')


In [10]:
tweets

,tweet
0,covid denier website next
1,scotland ha people hospital covid point pandem...
2,want pandemic end trying away arent helping u ...
3,world health organization say new covid xe var...
4,covid vaccine unvaccinated aggressively threat...
...,...
1995,billion waste mountain junk ppe bought tory cr...
1996,swedens twitter account covid stats hasnt upda...
1997,see exact sht went last year grammys people
1998,unemployment nation wa rising covid result gro...


In [ ]:
tweets.to_csv('covid_tweets.csv')

In [3]:
import json

credentials = json.load(open('credentials.json'))

consumer_key = credentials['consumer_key']
consumer_secret = credentials['consumer_secret']
access_token = credentials['access_token']
access_token_secret = credentials['access_token_secret']

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [36]:
def search_tweets(searchTerm):
    n = 1000
    tweets = tweepy.Cursor(api.search_tweets,q=searchTerm,lang="en",tweet_mode="extended").items(n)
    tweets_json=[]
    for tweet in tweets:
        tweets_json.append(tweet._json)
    return tweets_json

In [38]:
# covid_tweets = search_tweets('covid')
omicron_tweets = search_tweets('omicron')
# world_economy_tweets = search_tweets('world economy')
# remittance_tweets = search_tweets('remittance')

In [32]:
def tweets_to_csv(tweets_json, name='covid'):
    tweets = [tweet['full_text'] for tweet in tweets_json]
    tweet_df = pd.DataFrame({'tweet': tweets})
    tweet_df['tweet'] = tweet_df['tweet'].apply(p.clean)
    tweet_df.to_csv(f'{name}_tweets.csv', index=False)

In [33]:
# tweets_to_csv(covid_tweets, 'covid')
tweets_to_csv(omicron_tweets, 'omicron')
# tweets_to_csv(world_economy_tweets, 'world_economy')
# tweets_to_csv(remittance_tweets, 'remittance')

In [11]:
covid_tweets['_json']

TypeError: list indices must be integers or slices, not str